<a href="https://colab.research.google.com/github/FlaviusT/RecommenderSystem/blob/master/RecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS


In [64]:
!pip install scipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from scipy import spatial

# INITIALIZE DATA

In [69]:
# Import movies
movies = pd.read_csv('/content/datasets/movies.csv')

# Import ratings
ratings = pd.read_csv('/content/datasets/usersRatings.csv')

# Import users
users = pd.read_csv('/content/datasets/users.csv')

# MODEL-BASED ALGORITHMS

# -> kNN

In [93]:
def kNN(movies, movie_to_test, top_movies_number):
  closest_movies = []
  tmp_table = string_list_to_binary(movies)

  for index, movie in tmp_table.iterrows():
    if movie['titleId'] != movie_to_test['titleId']:
      distance = Similarity(movie_to_test, movie)
      closest_movies.append(movie['titleId'], distance)

  closest_movies.sort(key=operator.itemgetter(1))
  neighbors = []
    
  for neighbour_counter in range(top_movies_number):
      neighbors.append(closest_movies[neighbour_counter])

  return neighbors

# Compute the distance between movies(4 dimensions, by rating, genres, directors and cast)
def Similarity(movie1, movie2):
    rating_first_movie = movie1['imdbscore']
    rating_second_movie = movie2['imdbscore']
    rating_distance = spatial.distance.cosine(rating_first_movie, rating_second_movie)
    
    genres_first_movie = movie1['genres_binary']
    genres_second_movie = movie2['genres_binary']
    genre_distance = spatial.distance.cosine(genres_first_movie, genres_second_movie)

    directors_first_movie = movie1['directors_binary']
    directors_second_movie = movie2['directors_binary']
    directors_distance = spatial.distance.cosine(directors_first_movie, directors_second_movie)

    cast_first_movie = movie1['cast_binary']
    cast_second_movie = movie2['cast_binary']
    cast_distance = spatial.distance.cosine(cast_first_movie, cast_second_movie)
    
    return genre_distance#rating_distance + genre_distance + directors_distance + cast_distance

# This function transforms a string of values(like genres, cast, etc)
# into a list of binary values
def string_list_to_binary(movies):
  temp_movies_table = pd.DataFrame(columns = ['titleId','genres_binary', 'directors_binary', 'cast_binary'])

  all_unique_genres = get_unique_values('genres')
  all_unique_cast = get_unique_values('cast')
  all_unique_directors = get_unique_values('director')

  for index, movie in movies.iterrows():
    temp_movies_table = temp_movies_table.append({'titleId':movie['titleId'], 
                                                  'genres_binary':create_binary_values(movie['genres'], all_unique_genres), 
                                                  'directors_binary':create_binary_values(movie['director'], all_unique_directors), 
                                                  'cast_binary':create_binary_values(movie['cast'], all_unique_cast)}, ignore_index=True)
  return temp_movies_table
     
# This method get all unique values of a column from CSV file
def get_unique_values(column_name):
  values = movies[column_name]
  values_list = list()

  for val in values:
    values_list.append(val.split('|'))

  values_dataframe = pd.DataFrame(values_list)

  # Remove duplicate values to obtain all unique values from the CSV file
  return values_dataframe[0].unique()

def create_binary_values(values, unique_values):
  all_values = values.split('|')
  binaryList = []
  
  for val in unique_values:
    if val in all_values:
      binaryList.append(1)
    else:
      binaryList.append(0)

  return binaryList

In [99]:
import operator
dt = [[135328, 'Cesar del Alamo', 'Bruno Solo|Jean Louisa Kelly|Marta Ibanez Cruz|Javier Lago', 5.5, 'Action|Adventure|Comedy']]
d = pd.DataFrame(dt, columns = ['titleId', 'director', 'cast', 'imdbscore', 'genres'])
test_movie = string_list_to_binary(d).iloc[0]
print(kNN(movies, test_movie, 10))

titleId                                                        135328
genres_binary       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
directors_binary    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
cast_binary         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 0, dtype: object


KeyboardInterrupt: ignored